# Detection of Ransomware Attack Families via Bitcoin Transactions

This is a group project by Master 2 Data Science students at Ecole Polytechnique.

The data used in this study is courtesy of UC Irvine:

    Dua, D. and Graff, C. (2019). UCI Machine Learning Repository [http://archive.ics.uci.edu/ml]. Irvine, CA: University of California, School of Information and Computer Science.

<center><img src="./media/UCI.jpg"/></center>

## 1. Problem Statement

### Ransomware attacks

Starting as early as 1989 with the first documented ransomware known as the AIDS trojan, the use of ransomware scams has grown internationally. Governments worldwide saw a 1,885% increase in ransomware attacks, and the health care industry faced a 755% increase in those attacks in 2021.

A ransomware attack is a virus that infects a computer, a server or a storage device, by temporarily corrupting all of its files with a special encryption. The key to unlock this encryption is only available with the attacker. In exchange for this key, the attacker demands a certain amount of ransom, this is where the attack name derives from.

Over the past four years, the ransom demanded by hackers increased by a shocking 2,966.66 percent. In 2021, the average ransom demand reached **$ 220,298** — up 43 percent compared to 2020. The explosive growth in ransomware demand was in 2019, where the average ransom demand grew 14 times, up **from $6,000 in 2018 to $84,000 by the end of the year**.

### Ransom and Bitcoin &#x20BF;
These payments usually take several forms to be executed. For instance, ransomware attackers usually demand payment to be wired through Western Union or paid through a specialized text message. Some attackers demand payment in the form of gift cards like an Amazon or iTunes Gift Card. Recently, the attackers started asking the victims to purchase the bitcoins required to pay the ransom. The victim sends the money via a bitcoin exchange to the hacker's bitcoin wallet. The criminals confirm payment via email or a Tor site and, if the victim is lucky, will provide the means to decrypt the victim's files. The attackers took advantage of the anonymity provided by the different cryptocurrencies such as Bitcoin, Ethereum or Dogecoin. Even though all transactions are public by nature, user identification is not required to join the network.

People were and still are paying these ransoms due to the importance of their encrypted data. These numbers are even increasing over the years. Especially that the proportion of Cryptolocker (a ransomware) victims claim to have agreed to pay the ransom to recover their files (41%) seems to be much larger than expected (3% was conjectured by Symantec, 0.4% by Dell SecureWorks).

### Taking action
Many studies were conducted in order to tackle this problem and try to reduce its effects on users of the world wide web. The majority of these studies were based on pre-set filtering and pre-defined rules to decide whether a party is an attacker or not.The goal was to track the different ransomware attack families in order to limit their actions and proceed to label them as malicious.

In this challenge, the goal will be to create a model that is able to detect whether the given addresses belong to any of the attacker families by tracing the cryptocurrency transactions in the entire Bitcoin transaction graph from 2009 to 2018.

## 2. Data Exploration

The data used contains the entire Bitcoin transaction graph from 2009 January to 2018 December. Using a time interval of 24 hours, the dataset contains daily transactions on the network forming the Bitcoin graph. Network edges that transfer **less than &#x20BF;0.3** (read: 0.3 bitcoins) were filtered out since ransom amounts are rarely below this threshold.

We created a union of datasets from three widely adopted studies: Montreal, Princeton and Padua. The combined dataset contains **24,486 addresses** from **27 ransomware families**.

### Extracted features

We extract from the graph of Bitcoin transactions a number of features that are resampled on a 24-hour frame.

Using the graph topology, we define the following features:
<ul>
    <li>income</li>
    <li>neighbors</li>
    <li>weight</li>
    <li>length</li>
    <li>count</li>
    <li>loop</li>
</ul>

Let's explain how we get each feature and what do they mean. In order to do so, we must at first define the directed graph G = (V, E), where:

- V are the vertices that represent a user, in our case each user is represented by an address e.g. the hash of the user's nickname
- E are the edges that represent a part of a transaction, originating from an output address $a_{out}$ and is directed towards an input address $a_{in}$. 

**N.B.:** We note that a full transaction can contain more than one input address $a_{in}$, i.e. it can be formed of more than one edge originating from the same or from different output addresses $a_{out}$.

This structure of G is called an address-transaction structure, which represents all the transactions done between all the different users in the blockchain, in our case Bitcoin, between 2009 January and 2018 December.

Hence, we define the **income** of an address $u$ as the total amount of coins output to $u$, i.e. the sum of all the edges that have an input address $a_{in}$ equal to $u$.

Also, we denote the **neighbors** of an address $u$ the number of transactions which have $u$ as one of its output addresses $a_{out}$, i.e. the number of distinct addresses that u has sent money towards.

While the first two features were cumulative i.e. they can grow over time in the graph, the remaining features are extracted from the graph basing on the specific 24-hour interval.

To better understand how we extract these features from the graph, we need to introduce the concept of a *starter transaction*.

We consider a window $w$ of 24 hours interval, where we have a number of transactions that happen in each hour of this window $w$. We mark the timestamp of the earliest transaction done in this window as $t$. We define the set of all the output transactions that are originated before the timestamp $t$ as $TX_{out}$. In this case, we can name a transaction as a *starter transaction* if it does not contain any output transaction from the set $TX_{out}$. In other terms, the transaction does not contain an output transaction originating from the current window $w$.

To clarify the image, we give the following example of a window $w$ that contains more than one starter and non-starter transactions:

<center><img src="./media/graph_transactions.jpg"/></center>

Here, we have a toy network of 10 users (addresses) and 7 full transactions. The transactions tx1, tx3, tx4, and tx5 are starter transactions, whereas the other transactions are not. We notice the coin amounts that are shown on the edges, and that the sum of the input transactions is equal to the sum of the output transactions.

Now, we can continue defining the extracted features:

We denote **weight** of an address $u$ the sum of fraction of coins that originate from a starter transaction and reach $u$.

We also define the **loop** of an address $u$ which is the number of starter transactions which are connected to $u$ with more than one directed paths, e.g. in our toy network, the address $a_{10}$ has the loop equal to 1; this is because of the starter transaction tx4 that splits its coins, moves these coins in different paths, and then merge these coins in the same address $a_{10}$.

For the length and count features, we define a **chain** ending at $u$ as an acyclic directed path originating from any starter transaction and ending at address $u$.

So, the **length** of an address $u$ is the number of non-starter transactions on its longest chain. A length of zero implies that the address is an output address of a starter transaction.

Finally, the **count** of an address $u$ is the number of starter transactions which are connected to u through a chain.


### Loading required libraries

This cell should always contain ALL of our imports.

In [ ]:
import os
import warnings
import pickle
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import boxcox
from tqdm.notebook import tqdm

from sklearn.model_selection import train_test_split
import pdb
import datetime
from scipy.stats import pearsonr

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score, log_loss, plot_roc_curve, f1_score, precision_score, recall_score
from sklearn.model_selection import RandomizedSearchCV
import matplotlib
from prettytable import PrettyTable
from pprint import pprint

warnings.filterwarnings("ignore")

### Data load

We'll use the download_data script in order to download our BitcoinHeist data.


In [ ]:
!python download_data.py

### Loading the data

We have 2916697 examples in hand that we split into training and testing datasets as follows :


*   The training dataset contains 75% of the original dataset. It is accessible in CSV format under the train folder.

*   The testing dataset contains 25% of the original dataset. It is accessible in CSV format under the test folder.



The next function allows us to load the training and testing datasets.

In [ ]:
from glob import glob

def get_file_list_from_dir(*, path, datadir):
    data_files = sorted(glob(os.path.join(path, "data", datadir, "*.csv.gz")))
    return data_files

In [ ]:
#Loading the training dataset
train_files = get_file_list_from_dir(path=".", datadir="train")
dtrain = pd.concat((pd.read_csv(f) for f in train_files))

#Loading the testing dataset
test_files = get_file_list_from_dir(path=".", datadir="test")
dtest = pd.concat((pd.read_csv(f) for f in test_files))


### Dataset insights

In [ ]:
dtrain.head()

In [ ]:
dtrain.shape # the training dataframe has 2 187 523 rows and 10 columns

In [ ]:
dtest.head()

In [ ]:
dtest.shape # the testing dataframe has 729 174 rows and 10 columns

In [ ]:
print(dtrain.columns) #the names of the columns

Index(['address', 'year', 'day', 'length', 'weight', 'count', 'looped',
       'neighbors', 'income', 'label'],
      dtype='object')


In [ ]:
dtrain.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2916697 entries, 0 to 2916696
Data columns (total 10 columns):
 #   Column     Dtype  
---  ------     -----  
 0   address    object 
 1   year       int64  
 2   day        int64  
 3   length     int64  
 4   weight     float64
 5   count      int64  
 6   looped     int64  
 7   neighbors  int64  
 8   income     float64
 9   label      object 
dtypes: float64(2), int64(6), object(2)
memory usage: 222.5+ MB


In [ ]:
dtrain.describe()

In [ ]:
dtrain.describe(include=object) #so that we can see some descriptive statistices for the object columns as well

As we've seen from the few descriptions of the dataset above, some columns are of type float, some integer and some object. 

We'll now proceed to do some exploratory data analysis to have more insight on our dataset.

### Feature exploration

In this part we'll try to further understand the features in our dataset, mainly the different classes of the labels which is our target variable.

We'll also see if there are any missing values.

In [ ]:
#checking which columns contain NaN values
dtrain.isna().any()

address      False
year         False
day          False
length       False
weight       False
count        False
looped       False
neighbors    False
income       False
label        False
dtype: bool

In [ ]:
dtrain.year.unique() #the information that's available is from .. consecutive years from ... to ...


In [ ]:
dtrain.year.value_counts() #we have almost the same number of info for each year

In [ ]:
dtrain.looped.unique().shape


(10168,)

In [ ]:
dtrain.neighbors.unique().shape

(814,)

In [ ]:
dtrain['count'].unique().shape

(11572,)

We'll focus on our target variable "label" since for the features, there is a lot of variability of the values taken. Thus it won't be helpful to study their unique values. 

We can however see how their values change with regard to the class of the label.

In [ ]:
dtrain.label.unique().shape

(29,)

In [ ]:
dtrain.label.unique()

For the label, we have 29 different values which we can classify in 2 major categories. The white category and the ransomware category containig the rest of the values that are not white. 

In [ ]:
dtrain.loc[dtrain["label"]=="white", "label"].value_counts()

In [ ]:
dtrain.loc[dtrain["label"]!="white", "label"].value_counts()

In [ ]:
df1 = dtrain[dtrain["label"]!="white"]["label"]

In [ ]:
len(df1)

We have .... white labels vs .... non-white lables => imbalanced data.

This should be taken into consideration when choosing
the performance evaluation metric.

In [ ]:
df2 = dtrain[dtrain["label"]!="white"]["year"]

In [ ]:
df2.value_counts() 

We can see that the number of Ransomware varies greatly from one year to another. There are years where it is much greater compared to the rest. This can be seen in the case of the year 2016 where the number of Ransomware peaked.

In [ ]:
df2 = dtrain[dtrain["label"]!="white"]["neighbors"]


In [ ]:
df2.value_counts() #most of the ransomware cases are when the value of neighbors is either 2 or 1

### Feature by feature exploration

Mootez, Rihem

In [2]:
...

Ellipsis

### Classes exploration

bourhan

## 3. Workflow

Issa, Joel

## 4. Ramp Tutorial

Ronny